In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sbn
import scipy.stats as stats
from pandas.plotting import scatter_matrix
import statsmodels.api as sm
import datetime as dt
import pandasql as ps

In [2]:
def change_cols_to_floats(dataframe,lst):
    
    for i in lst:
        dataframe[i] = dataframe[i].str.replace(',', '')
        dataframe[i] = dataframe[i].astype(float)
    return dataframe
def make_date_time_col(df):
    df['Hour Number'] = df_total['Hour Number'].replace(24, 0)
    df['Hour Number'] = df_total['Hour Number'].replace(25, 0)
    df['Data Date']= df['Data Date'].astype(str)
    df['Hour Number'] = df['Hour Number'].astype(str)
    df['New_datetime'] = df['Data Date'].map(str) + " " + df['Hour Number']
    df['Hour Number'] = df['Hour Number'].astype(int)
    
    return df

def make_hourly_demand_means(df,lst):
    d = {}
    for i in lst:
        filt =df['Hour Number']==i
        d[i] = df.loc[filt]['Demand (MW)'].mean()
    return d

def graph_maker_for_energy_type_by_hour(df,column, lst = np.arange(0,24)):
    
    d= {}
    for i in lst:
        filt =df['Hour Number']==i
        hour_avg = df.loc[filt][column].mean()
        d[i]=hour_avg
    x = d.keys()
    y = d.values()
    fig, ax =plt.subplots(figsize = (8,8))
    ax.plot(x, y)
    ax.set_title(column)
    ax.set_xlabel('Time (Hours in Day)')
    ax.set_xticks(lst)
    
    
    plt.show()

In [3]:
def temp_column_convert_float2(pd_series):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0F','')
        
        lst.append(string)
    
    results = pd.Series(lst)
    return results 

def press_column_convert_float2(pd_series):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0in','')
        if string == '0.00\xa0':
            string = '0.00'
        lst.append(string)
    
    results = pd.Series(lst)
#     results2 = results.astype(float)
    return results

def wind_column_convert(pd_series):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0mph','')
        if string == '0.00\xa0':
            string = '0.00'
        lst.append(string)
    
    results = pd.Series(lst)
#     results2 = results.astype(float)
    return results

def dew_column_convert_float2(pd_series):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0F','')
        
        lst.append(string)
    
    results = pd.Series(lst)
    return results 

In [4]:
def change_cols_to_floats2(dataframes_lst):
    floated_dfs =[]
    for df in dataframes_lst:
        lst_cols = ['Demand (MW)','Net Generation (MW) from Natural Gas', 'Net Generation (MW) from Nuclear',
                    'Net Generation (MW) from All Petroleum Products',
                    'Net Generation (MW) from Hydropower and Pumped Storage', 
                    'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind', 
                    'Net Generation (MW) from Other Fuel Sources','Net Generation (MW)',
                    'Demand Forecast (MW)', 'Total Interchange (MW)', 
                    'Net Generation (MW) (Adjusted)','Net Generation (MW) from Coal',
                    'Sum(Valid DIBAs) (MW)','Demand (MW) (Imputed)',
                    'Net Generation (MW) (Imputed)','Demand (MW) (Adjusted)']
        for i in lst_cols:
            df[i] = df[i].str.replace(',', '')
            df[i] = df[i].astype(float)
            floated_dfs.append(df)
        result = pd.concat(floated_dfs)
    return result

In [5]:
data = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jan_Jun.csv')
data_2 = pd.read_csv('~/Downloads/EIA930_BALANCE_2020_Jul_Dec.csv')

data = data[data['Balancing Authority'] == 'ERCO']
data_2 = data[data['Balancing Authority'] == 'ERCO']

/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,15,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11,14,16,17,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_total = change_cols_to_floats2([data, data_2])
# data_2_convert = change_cols_to_floats(data_2)

In [7]:
def make_date_time_col2(df):
    
    df['Hour Number'] = df_total['Hour Number'].replace(24, 0)
    df['Hour Number'] = df_total['Hour Number'].replace(25, 0)
    df['Data Date']= df['Data Date'].astype(str)
    df['Hour Number'] = df['Hour Number'].astype(str)
    df['New_datetime'] = df['Data Date'].map(str) + " " + df['Hour Number']
    df['Hour Number'] = df['Hour Number'].astype(int)
    df['New_datetime']= df['New_datetime'].apply(lambda x:f'{x}:00:00')
    df['New_datetime'] = pd.to_datetime(df['New_datetime'],infer_datetime_format=True, format ='%m/%d/%Y %H')
    df['Demand Delta'] = df['Demand Forecast (MW)']- df['Demand (MW)']
    df['Net Generation Delta'] = df['Net Generation (MW)']- df['Demand (MW)']
    print('done')
    return df

In [8]:
def drop_extra_cols(df_texas):
    del df_texas['UTC Time at End of Hour']
    del df_texas['Balancing Authority']
    del df_texas['Net Generation (MW) (Imputed)']
    del df_texas['Demand (MW) (Imputed)']
    del df_texas['Net Generation (MW) from All Petroleum Products']
    del df_texas['Net Generation (MW) from Unknown Fuel Sources']
    del df_texas['Data Date']
    del df_texas['Hour Number']
    del df_texas['Local Time at End of Hour']
    return df_texas
    

In [9]:
df_texas = make_date_time_col2(df_total)

done


In [10]:
def make_hourly_demand_means2(df,lst=np.arange(0,24)):
    
    d = {}
    for i in lst:
        filt =df['Hour Number']==i
        d[i] = df.loc[filt]['Demand (MW)'].mean()
    return d

In [11]:
df_texas1=  drop_extra_cols(df_texas)

In [12]:
df_texas1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148478 entries, 66924 to 71290
Data columns (total 17 columns):
 #   Column                                                  Non-Null Count   Dtype         
---  ------                                                  --------------   -----         
 0   Demand Forecast (MW)                                    148478 non-null  float64       
 1   Demand (MW)                                             148478 non-null  float64       
 2   Net Generation (MW)                                     148478 non-null  float64       
 3   Total Interchange (MW)                                  148478 non-null  float64       
 4   Sum(Valid DIBAs) (MW)                                   148478 non-null  float64       
 5   Demand (MW) (Adjusted)                                  148478 non-null  float64       
 6   Net Generation (MW) (Adjusted)                          148478 non-null  float64       
 7   Net Generation (MW) from Coal               

# READ IN WEATHER CSV

In [13]:
df_dallas =pd.read_csv('/Users/cp/Desktop/capstone2/DALLASV2_FINAL_WEATHER.csv')

In [14]:
df_dallas['New_datetime'] = pd.to_datetime(df_dallas['New_datetime'],infer_datetime_format=True,format ='%m/%d/%Y %H')

In [15]:
Energy_Houston_weather=df_texas.merge(df_dallas, left_on ='New_datetime', right_on='New_datetime' )

In [16]:
Energy_Houston_weather['Cloud_numerical'] =  Energy_Houston_weather['cloud']

In [17]:
d1 =  {
    'Fair':0
    ,'Mostly Cloudy':2
    ,'Cloudy':1
    ,'Partly Cloudy':1
    ,'Light Rain':2
    , 'Light Drizzle':2
    ,'Rain':2
    ,'Light Rain with Thunder':2
    ,'Heavy T-Storm':2
    ,'Thunder':2
    , 'Heavy Rain':2
    ,'T-Storm':2
    , 'Fog':2
    , 'Mostly Cloudy / Windy':2
    , 'Cloudy / Windy':2
    , 'Haze':1
    , 'Fair / Windy':0
    , 'Partly Cloudy / Windy':1
    , 'Light Rain / Windy':2
    , 'Heavy T-Storm / Windy':2
    , 'Heavy Rain / Windy':2
    , 'Widespread Dust':1
    , 'Thunder and Hail':2
    ,'Thunder / Windy':2
    ,'Blowing Dust':1
    , 'Patches of Fog':1
    , 'Blowing Dust / Windy':1
    , 'Rain / Windy':2
    , 'Fog / Windy':2
    , 'Light Drizzle / Windy':2
    , 'Haze / Windy':1
    ,'Light Snow / Windy':1
    , 'Light Snow':1
    ,'T-Storm / Windy':2
    ,'Light Sleet':1
}

In [18]:
Energy_Houston_weather['Cloud_numerical'].replace(d1, inplace= True)
# Energy_Houston_weather['Cloud_numerical'].astype(float)

In [19]:
Energy_Houston_weather['humidity'].unique()

array(['46\xa0%', '67\xa0%', '64\xa0%', '40\xa0%', '37\xa0%', '39\xa0%',
       '41\xa0%', '53\xa0%', '51\xa0%', '48\xa0%', '38\xa0%', '42\xa0%',
       '47\xa0%', '57\xa0%', '62\xa0%', '89\xa0%', '69\xa0%', '75\xa0%',
       '77\xa0%', '74\xa0%', '84\xa0%', '83\xa0%', '65\xa0%', '97\xa0%',
       '90\xa0%', '93\xa0%', '86\xa0%', '80\xa0%', '55\xa0%', '36\xa0%',
       '45\xa0%', '54\xa0%', '58\xa0%', '59\xa0%', '50\xa0%', '73\xa0%',
       '70\xa0%', '79\xa0%', '76\xa0%', '63\xa0%', '43\xa0%', '35\xa0%',
       '34\xa0%', '33\xa0%', '31\xa0%', '32\xa0%', '66\xa0%', '52\xa0%',
       '61\xa0%', '68\xa0%', '26\xa0%', '23\xa0%', '24\xa0%', '25\xa0%',
       '30\xa0%', '60\xa0%', '21\xa0%', '20\xa0%', '16\xa0%', '17\xa0%',
       '29\xa0%', '72\xa0%', '49\xa0%', '22\xa0%', '44\xa0%', '87\xa0%',
       '78\xa0%', '81\xa0%', '82\xa0%', '85\xa0%', '71\xa0%', '96\xa0%',
       '94\xa0%', '92\xa0%', '27\xa0%', '28\xa0%', '56\xa0%', '88\xa0%',
       '100\xa0%', '19\xa0%', '14\xa0%', '15\xa0%',

In [20]:
def column_convert_float(pd_series):
    lst = []
    for i in pd_series:
        lst1 = i.split('\\')
        
        lst.append(lst1[0])
    
    results = pd.Series(lst)
    return results 

def temp_column_convert_float2(pd_series, df= Energy_Houston_weather):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0F','')
        
        lst.append(string)
    
    results = pd.Series(lst)
    df['temp1']= results
    df['temp1'] = df['temp1'].astype(float)
    return df

def press_column_convert_float2(pd_series, df= Energy_Houston_weather):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0in','')
        if string == '0.00\xa0':
            string = '0.00'
        lst.append(string)
    
    results = pd.Series(lst)

    df['pressure1']= results
    df['pressure1'] = df['pressure1'].astype(float)
    return df

def wind_column_convert(pd_series, df= Energy_Houston_weather):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0mph','')
        if string == '0.00\xa0':
            string = '0.00'
        lst.append(string)
    
    results = pd.Series(lst)
#     results2 = results.astype(float)
    df['wind1']= results
    df['wind1'] = df['wind1'].astype(float)
    return df

def dew_column_convert_float2(pd_series, df= Energy_Houston_weather):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0F','')
        
        lst.append(string)
    
    results = pd.Series(lst)
    df['dew1']= results
    df['dew1'] = df['dew1'].astype(float)
    return df

def humidity_column_convert_float2(pd_series, df= Energy_Houston_weather):
    lst = []
    for string in pd_series:
        string = string.replace(u'\xa0%','')
        
        lst.append(string)
    
    results = pd.Series(lst)
    df['humdity1']= results
    df['humdity1'] = df['humdity1'].astype(float)
    
    
    
    return df



In [21]:
Energy_Houston_weather = humidity_column_convert_float2(Energy_Houston_weather['humidity'])

In [22]:
Energy_Houston_weather= press_column_convert_float2(Energy_Houston_weather['pressure'])

In [23]:
Energy_Houston_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180472 entries, 0 to 180471
Data columns (total 29 columns):
 #   Column                                                  Non-Null Count   Dtype         
---  ------                                                  --------------   -----         
 0   Demand Forecast (MW)                                    180472 non-null  float64       
 1   Demand (MW)                                             180472 non-null  float64       
 2   Net Generation (MW)                                     180472 non-null  float64       
 3   Total Interchange (MW)                                  180472 non-null  float64       
 4   Sum(Valid DIBAs) (MW)                                   180472 non-null  float64       
 5   Demand (MW) (Adjusted)                                  180472 non-null  float64       
 6   Net Generation (MW) (Adjusted)                          180472 non-null  float64       
 7   Net Generation (MW) from Coal                  

In [24]:
Energy_Houston_weather= dew_column_convert_float2(Energy_Houston_weather['dew'])

In [25]:
Energy_Houston_weather = temp_column_convert_float2(Energy_Houston_weather['temp'])

In [27]:
Energy_Houston_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180472 entries, 0 to 180471
Data columns (total 31 columns):
 #   Column                                                  Non-Null Count   Dtype         
---  ------                                                  --------------   -----         
 0   Demand Forecast (MW)                                    180472 non-null  float64       
 1   Demand (MW)                                             180472 non-null  float64       
 2   Net Generation (MW)                                     180472 non-null  float64       
 3   Total Interchange (MW)                                  180472 non-null  float64       
 4   Sum(Valid DIBAs) (MW)                                   180472 non-null  float64       
 5   Demand (MW) (Adjusted)                                  180472 non-null  float64       
 6   Net Generation (MW) (Adjusted)                          180472 non-null  float64       
 7   Net Generation (MW) from Coal                  

In [30]:
len(Energy_Houston_weather['Demand Delta'].unique())

2721

In [31]:
def drop_extra_cols2(df):
    del df['Demand Forecast (MW)']
    del df['Demand (MW)']
    del df['temp']
    del df['dew']
    del df['humidity']
    del df['wind_speed']
    del df['pressure']
    del df['precip']
    del df['cloud']
    
    del df['time_rounded4']
    del df['New_datetime2']
    
    return df

In [32]:
Energy_Houston_weather = drop_extra_cols2(Energy_Houston_weather)

In [33]:
Energy_Houston_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180472 entries, 0 to 180471
Data columns (total 20 columns):
 #   Column                                                  Non-Null Count   Dtype         
---  ------                                                  --------------   -----         
 0   Net Generation (MW)                                     180472 non-null  float64       
 1   Total Interchange (MW)                                  180472 non-null  float64       
 2   Sum(Valid DIBAs) (MW)                                   180472 non-null  float64       
 3   Demand (MW) (Adjusted)                                  180472 non-null  float64       
 4   Net Generation (MW) (Adjusted)                          180472 non-null  float64       
 5   Net Generation (MW) from Coal                           180472 non-null  float64       
 6   Net Generation (MW) from Natural Gas                    180472 non-null  float64       
 7   Net Generation (MW) from Nuclear               